# Notebook to evaluate a trained model with pyfolio
Note: pyfolio must be install using pip install git+https://github.com/quantopian/pyfolio. There are known bugs with pip install pyfolio.

In [40]:
import os
import glob

import pandas as pd
import pyfolio

from deep_rl_asset_allocation.configs import paths_config, data_config
from deep_rl_asset_allocation.utils import data_loader_utils
import deep_rl_asset_allocation.preprocessing.data_preprocessing as data_preprocessing

In [41]:
import warnings
warnings.filterwarnings('ignore')

In [42]:
# plotting in notebook
%matplotlib inline

### Helper Functions

In [43]:
def get_daily_return_as_df(df):
    df['daily_return'] = df.account_value.pct_change(1)
    return df


def get_daily_return_as_series(df):
    df1 = df.copy()
    df1['Date'] = pd.to_datetime(df1['Date'])
    df1.set_index('Date', drop=False, inplace=True)
    df1.index = df1.index.tz_localize('UTC')
    del df1['Date']
    ts = pd.Series(df1['daily_return'].values, index=df1.index)
    return ts

def get_training_results_account_value(df_unique_trade_dates, results_csv_dir=paths_config.results_csv_dir):
    df_account_value = pd.DataFrame()
    filepath = os.path.join(results_csv_dir, "test", f'portfolio_value_*.csv')
    num_csv_results_files = len(glob.glob(filepath))
    for idx in range(0, num_csv_results_files):
        csv_results_file = os.path.join(results_csv_dir, "test", f'portfolio_value_{idx}.csv')
        print(f"csv_results_file: {csv_results_file}")
        df = pd.read_csv(csv_results_file)
        df_account_value = df_account_value.append(df, ignore_index=True)
    # change the column name
    df_account_value = pd.DataFrame({'account_value': df_account_value['0']})
    # merge with unique trade dates from 2016-01-04 or index 64
    df_account_value = df_account_value.join(df_unique_trade_dates.reset_index(drop=True))
    df_account_value = df_account_value[['Date', 'account_value']]
    return df_account_value

### DJIA: Out-of-sample Test Data

In [44]:
# load DJIA - out of sample data (test data)
df_djia_out_of_sample = pd.read_csv(paths_config.TESTING_DATA_FILE)
df_djia_out_of_sample["Date"] = df_djia_out_of_sample.apply(data_preprocessing.convert_datadate_to_datetime, axis=1)

# get data between dates
df_djia_out_of_sample = df_djia_out_of_sample[(df_djia_out_of_sample['Date'] >= data_config.TESTING_START) & (df_djia_out_of_sample['Date'] <= data_config.TESTING_END)]
df_djia_out_of_sample = df_djia_out_of_sample.reset_index(drop=True)

In [45]:
# get daily return as relative difference in adjusted close price of DJIA
df_djia_out_of_sample['daily_return'] = df_djia_out_of_sample['Adj Close'].pct_change(1)

df_djia_out_of_sample

,Date,Open,High,Low,Close,Adj Close,Volume,daily_return
0,2016-01-04,17405.480469,17405.480469,16957.630859,17148.939453,17148.939453,148060000,NaN
1,2016-01-05,17147.500000,17195.839844,17038.609375,17158.660156,17158.660156,105750000,0.000567
2,2016-01-06,17154.830078,17154.830078,16817.619141,16906.509766,16906.509766,120250000,-0.014695
3,2016-01-07,16888.359375,16888.359375,16463.630859,16514.099609,16514.099609,176240000,-0.023211
4,2016-01-08,16519.169922,16651.890625,16314.570312,16346.450195,16346.450195,141850000,-0.010152
...,...,...,...,...,...,...,...,...
1506,2021-12-27,35954.480469,36306.609375,35954.480469,36302.378906,36302.378906,244350000,0.009786
1507,2021-12-28,36302.988281,36527.261719,36302.988281,36398.210938,36398.210938,239090000,0.002640
1508,2021-12-29,36421.140625,36571.550781,36396.191406,36488.628906,36488.628906,213480000,0.002484
1509,2021-12-30,36522.480469,36679.441406,36372.128906,36398.078125,36398.078125,205620000,-0.002482


In [46]:
series_djia_out_of_sample = get_daily_return_as_series(df_djia_out_of_sample)
series_djia_out_of_sample

Date
2016-01-04 00:00:00+00:00         NaN
2016-01-05 00:00:00+00:00    0.000567
2016-01-06 00:00:00+00:00   -0.014695
2016-01-07 00:00:00+00:00   -0.023211
2016-01-08 00:00:00+00:00   -0.010152
                               ...   
2021-12-27 00:00:00+00:00    0.009786
2021-12-28 00:00:00+00:00    0.002640
2021-12-29 00:00:00+00:00    0.002484
2021-12-30 00:00:00+00:00   -0.002482
2021-12-31 00:00:00+00:00   -0.001642
Length: 1511, dtype: float64

### Load Training Results: Account Value

In [47]:
print(data_config.TESTING_START)
print(data_config.TESTING_END)

2016-01-04
2022-01-01


In [48]:
# load Dow 30 - in sample data (training data)
df_djia_in_sample = data_loader_utils.load_preprocessed_djia_data()
df_djia_in_sample = data_loader_utils.get_data_between_dates(df_djia_in_sample, start=data_config.TESTING_START, end=data_config.TESTING_END)

# rebalance_window is the number of months to retrain the model
rebalance_window = data_config.REBALANCE_WINDOW
# validation_window is the number of months to validation the model and select for trading
validation_window = data_config.VALIDATION_WINDOW

# get unique trade dates to load model results
unique_trade_dates = data_loader_utils.get_data_between_dates(df_djia_in_sample, start=data_config.VALIDATION_START, end=data_config.TESTING_END).Date.unique()
df_unique_trade_dates = pd.DataFrame({'Date': unique_trade_dates})

# filter for when Testing Starts
df_unique_trade_dates = df_unique_trade_dates[df_unique_trade_dates.Date >= data_config.TESTING_START]
df_unique_trade_dates.index = df_unique_trade_dates.Date.factorize()[0]
df_unique_trade_dates

Found prevouisly saved pre-processed data: /Users/Aidan.Keaveny/git/deep-rl-asset-allocation/deep_rl_asset_allocation/data/csv/preprocessed_djia_data.csv


,Date
0,2016-01-04
1,2016-01-05
2,2016-01-06
3,2016-01-07
4,2016-01-08
...,...
1506,2021-12-27
1507,2021-12-28
1508,2021-12-29
1509,2021-12-30


In [49]:
results_dir = os.path.join(paths_config.parent_dir, 'results')
results_trained_models_dir = os.path.join(results_dir, 'trained_models')
results_figs_dir = os.path.join(results_dir, 'figs')
results_csv_dir = os.path.join(results_dir, 'csv')

df_training_results_account_value = get_training_results_account_value(df_unique_trade_dates, results_csv_dir)
# df_training_results_account_value.account_value.plot()
df_training_results_account_value

csv_results_file: /Users/Aidan.Keaveny/git/deep-rl-asset-allocation/deep_rl_asset_allocation/results/csv/test/portfolio_value_0.csv
csv_results_file: /Users/Aidan.Keaveny/git/deep-rl-asset-allocation/deep_rl_asset_allocation/results/csv/test/portfolio_value_1.csv
csv_results_file: /Users/Aidan.Keaveny/git/deep-rl-asset-allocation/deep_rl_asset_allocation/results/csv/test/portfolio_value_2.csv
csv_results_file: /Users/Aidan.Keaveny/git/deep-rl-asset-allocation/deep_rl_asset_allocation/results/csv/test/portfolio_value_3.csv
csv_results_file: /Users/Aidan.Keaveny/git/deep-rl-asset-allocation/deep_rl_asset_allocation/results/csv/test/portfolio_value_4.csv
csv_results_file: /Users/Aidan.Keaveny/git/deep-rl-asset-allocation/deep_rl_asset_allocation/results/csv/test/portfolio_value_5.csv
csv_results_file: /Users/Aidan.Keaveny/git/deep-rl-asset-allocation/deep_rl_asset_allocation/results/csv/test/portfolio_value_6.csv
csv_results_file: /Users/Aidan.Keaveny/git/deep-rl-asset-allocation/deep_rl_

,Date,account_value
0,2016-01-04,1.000000e+06
1,2016-01-05,1.000184e+06
2,2016-01-06,9.982776e+05
3,2016-01-07,9.944783e+05
4,2016-01-08,9.926858e+05
...,...,...
1444,2021-09-28,2.767699e+06
1445,2021-09-29,2.768247e+06
1446,2021-09-30,2.743795e+06
1447,2021-10-01,2.711248e+06


In [50]:
# get daily return as relative difference in account value
df_training_results_account_value['daily_return'] = df_training_results_account_value.account_value.pct_change(1)

# compare dates with test date
df_training_results_account_value['Date'] = df_djia_out_of_sample['Date']
df_training_results_account_value = df_training_results_account_value[['Date', 'account_value', 'daily_return']]

df_training_results_account_value

,Date,account_value,daily_return
0,2016-01-04,1.000000e+06,NaN
1,2016-01-05,1.000184e+06,0.000184
2,2016-01-06,9.982776e+05,-0.001906
3,2016-01-07,9.944783e+05,-0.003806
4,2016-01-08,9.926858e+05,-0.001802
...,...,...,...
1444,2021-09-28,2.767699e+06,-0.013305
1445,2021-09-29,2.768247e+06,0.000198
1446,2021-09-30,2.743795e+06,-0.008833
1447,2021-10-01,2.711248e+06,-0.011862


In [51]:
series_training_results_account_value = get_daily_return_as_series(df_training_results_account_value)  # [0:1097]
series_training_results_account_value

Date
2016-01-04 00:00:00+00:00         NaN
2016-01-05 00:00:00+00:00    0.000184
2016-01-06 00:00:00+00:00   -0.001906
2016-01-07 00:00:00+00:00   -0.003806
2016-01-08 00:00:00+00:00   -0.001802
                               ...   
2021-09-28 00:00:00+00:00   -0.013305
2021-09-29 00:00:00+00:00    0.000198
2021-09-30 00:00:00+00:00   -0.008833
2021-10-01 00:00:00+00:00   -0.011862
2021-10-04 00:00:00+00:00   -0.009441
Length: 1449, dtype: float64

### Pyfolio

In [52]:
with pyfolio.plotting.plotting_context(font_scale=1.1):
    # pyfolio.create_full_tear_sheet(returns=series_djia_out_of_sample, set_context=False)
    pyfolio.create_full_tear_sheet(returns=series_training_results_account_value, benchmark_rets=series_djia_out_of_sample, set_context=False)

Start date,2016-01-04
End date,2021-10-04
Total months,69
,Backtest
Annual return,18.746%
Cumulative returns,168.565%
Annual volatility,17.286%
Sharpe ratio,1.08
Calmar ratio,1.08
Stability,0.95
Max drawdown,-17.356%


Worst drawdown periods,Net drawdown in %,Peak date,Valley date,Recovery date,Duration
0,17.36,2019-12-17,2020-03-16,2020-04-16,88
1,17.18,2018-11-08,2018-12-24,2019-06-10,153
2,13.84,2018-01-29,2018-03-23,2018-09-21,170
3,11.88,2020-11-16,2021-03-04,2021-08-03,187
4,9.37,2021-08-20,2021-10-04,NaT,NaN
